In [1]:
import os
import datetime
import polars as pl
import pandas as pd
pl.Config(tbl_rows=100)
from src.najdi_rok import najdi_rok
from src.pocet_stran import pocet_stran
from src.bez_bordelu import bez_bordelu
from src.alt_friendly import alt_friendly
from src.zjisti_vazbu import zjisti_vazbu

In [3]:
df = pl.read_parquet(os.path.join("data/cnb_sloupce","100.parquet"))
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","leader.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","008.parquet")), left_on="001", right_on="001", how="left")
df = df.to_pandas()
df = df[df["leader"].str[6].isin(["a", "t"])]
df = df[~df["leader"].str[7].isin(["b", "i", "s", " "])]
df = df[(df["008"].str[15:17] == "xr") & (df["008"].str[35:38] == "cze")]
df = pl.from_pandas(df)
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","020.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","022.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","245.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","300.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","655.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","700.parquet")), left_on="001", right_on="001", how="left")
df = df.explode("022_a").filter(pl.col("022_a").is_null())
df = df.with_columns(pl.col('008').map_elements(najdi_rok, return_dtype=int).alias('rok'))
df = df.with_columns(pl.col('300_a').map_elements(pocet_stran, return_dtype=int).alias('stran'))
df = df.with_columns(pl.col('245_a').map_elements(bez_bordelu, return_dtype=str))
df = df.explode("020_q").with_columns(pl.col("020_q").map_elements(zjisti_vazbu, return_dtype=str).alias('vazba'))
df = df.explode('245_p').with_columns(pl.col('245_p').map_elements(bez_bordelu, return_dtype=str))
print(len(df))
df = df.filter(pl.col("rok") >= 1800)

df = df.filter((~pl.col("245_h").str.contains("grafika")) | pl.col("245_h").is_null()).unique(subset=["008","100_a","245_a","245_p"], keep="first")
print(len(df))

794838
705250


In [4]:
import requests
from bs4 import BeautifulSoup

## Le Monde's 100 Books of the Century

In [12]:
r = requests.get("https://en.wikipedia.org/wiki/Le_Monde%27s_100_Books_of_the_Century")
soup = BeautifulSoup(r.text)
tabulka = soup.find(class_="sortable")

In [42]:
for radek in tabulka.find_all("tr"):
    try:
        autor = radek.find_all("td")[2].text.strip().split(' ')[-1]
        print(f"{autor}: {radek.find_all("td")[1].text.strip()}")
        print(', '.join(set(df.filter(pl.col('100_a').str.contains(autor)).select(pl.col('245_a')).to_series().to_list())))
        print('***')
    except:
        pass

Camus: The StrangerThe Outsider
Spravedliví, Pád, Zápisníky I, Zápisníky II, Král Roman, Romány a povídky, Mýtus o Sisyfovi, Léto, Člověk revoltující, Cizinec, Eseje, Šťastná smrt, Exil a království, Mor, Nedorozumění, Velká výměna, Caligula, Zápisníky III, První člověk, Vyprávění severoamerických indiánů, Svět Židů
***
Proust: In Search of Lost TimeRemembrance of Things Past
Swannova láska, Jean Santeuil, Vzpomínky, Hledání ztraceného času, Dnové četby, Myšlenky, Eseje, Radosti a dny
***
Kafka: The Trial
Povídky, Moře, Tajemství zlatodolu, Levné obědy a večeře, Střední Povltaví a středočeské žulové pohoří, Střední Polabí, Život naživo, Vincenz Janke, Popis jednoho zápasu, Technika ve výkladní skříni, Člověk zítřka - kultura rozumu a vůle, Franz Kafka, Vcházím do pokoje a vidím-, Levné pokrmy zeleninové, Kvaše, Geologie a nauka o ložiskách, Pediatrická chirurgie, Pediatrická chirurgie 1963, Proces o hubičku, Noční dobrodružství, V krajích věčného ledu, Největší sluha člověka - slon, Ja

## Bokklubben World Library

In [47]:
r = requests.get("https://en.wikipedia.org/wiki/Bokklubben_World_Library")
soup = BeautifulSoup(r.text)
tabulka = soup.find(class_="sortable")

In [51]:
for radek in tabulka.find_all("tr"):
    try:
        autor = radek.find_all("td")[1].text.strip().split(' ')[-1]
        print(f"{autor}: {radek.find_all("td")[0].text.strip()}")
        print(', '.join(set(df.filter(pl.col('100_a').str.contains(autor)).select(pl.col('245_a')).to_series().to_list())))
        print('***')
    except:
        pass

Achebe: Things Fall Apart
Svět se rozpadá, Muž z lidu, Už nikdy klid
***
Andersen: Fairy Tales
Ošklivé káčátko, Kočky útočí, H. Ch. Andersen, úplný soubor jeho pohádek a povídek, J.C. Andersena vybrané pohádky, povídky a báchorky pro mládež a přátele její, Nesnadné rozhodnutí, Vybrané báchorky, Mořská žínka, Malá mořská víla, Pohádka mého života, Muži už jsou takoví, George a Laura, Tajný život v Bílém domě, Volám tě zpátky, Úplný soubor jeho pohádek a povídek, Sviňáček a jiné pohádky, Štěstí, Hans Christian Andersen: Nejkrásnější pohádky, Dospělost je mýtus, Ubulená hromádka štěstí, Hrášková princezna, Ďábel loterie, Nehezké káče a jiné povídky, Ošklivé kačátko, Císařovy nové šaty, Andersenovy báchorky, Pobřeží dětství, Andersenovy pohádky, Švéd loterista a jiné povídky, Jack a Jackie Kennedyovi, Osudná závěť, Křesadlo, Pastýřka a kominíček, Kreslené pohádky, Pasáček vepřů, Vlček trosečníkem, Císařův slavík, Sněhová královna a jiné pohádky, Máš modrou krev, Pohádky H. Ch. Andersena, B